In [308]:
#libraries that will be used in this HW 
import os
import shutil as sh
import pandas as pd
#you can add other libraries as needed
import datetime
import matplotlib.pylab as plt
%matplotlib inline 

#copy and pasted function from HW2
def q4():
    #unpack .csv data
    sh.unpack_archive(datafolder+'hw3_data2.zip')
    #list of all states in the the continental US
    cont_us=['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Idaho',\
             'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',\
             'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey',\
             'New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania',\
             'Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia',\
             'Washington','West Virginia','Wisconsin','Wyoming']
   
    #create a list of all relevant .csv files
    cont_us_files=[]
    for i in cont_us:
        cont_us_files.append(i+'.csv')
        
    result = {}

    for state in cont_us_files: #for each state file (e.g., 'Alabama.csv')
        f = open(datafolder+'hw3_data2/'+state,"r")
        all_lines = f.readlines()
        f.close()
       
        out={}
       
        for line in all_lines:
            if line.split(',')[4] != 'TOTAL':
                year = int(line.split(',')[5])
                if line.split(',')[4] != '':
                    number = float(line.split(',')[4])
                else:
                    number = 0  
                out[year]=number
        for key in out:
            if key in result:
                result[key]+= out[key]
            else:
                result[key] = out[key]
               
    return result


#Q1
#Starting point
def q1():
    f = plt.figure()
    #Your plotting code here
    bank_dict = q4()
    srtd_dict = sorted(bank_dict.items()) 
    x,y = zip(*srtd_dict)
    plt.plot(x,y)
    plt.xlabel('Year')
    plt.ylabel('Number of Banks')
    plt.show()
    #When saving the file to pdf assume that the output folder already exists
    f.savefig(outputfolder+'mart2336_q1.pdf') #<--- change username to your username

   
    
#Q2
#Starting point
def q2():
    #loading unrate data and setting date to date format and to index
    unrate = pd.read_csv(datafolder+'UNRATE.csv')
    unrate['DATE'] = pd.to_datetime(unrate['DATE'])
    unrate.set_index('DATE', inplace = True)
    unrate = unrate.loc['1950-01-01':'2010-10-01',:]
    
    #loading usrec data and setting date to date format and to index
    usrec = pd.read_csv(datafolder+'USREC.csv')
    usrec['DATE'] = pd.to_datetime(usrec['DATE'])
    usrec.set_index('DATE', inplace = True) 
    
    #concatinating data from unrate and usrec dataframes and adding column of current date and shifted date 
    combo = pd.concat([unrate,usrec],axis=1, join='inner')
    combo['end'] = combo.index
    combo['start'] = combo.end.shift(1)
    
    #plotting date against unrate data, adding labels, and adding source of data
    plt.figure(figsize=(10,5))
    plt.plot(unrate.index, unrate.UNRATE, linewidth=1, label='Unemployment Rate')
    plt.xlabel('Date', fontsize='x-large')
    plt.ylabel('Unemployment Rate', fontsize='x-large')
    plt.annotate('[Data from NBER]', (0,0), (-80,-20), fontsize=20, 
             xycoords='axes fraction', textcoords='offset points', va='top')
    
    #plotting shaded areas to indicate times of recession
    for i in combo.index:
        if combo.USREC[i]:
            plt.axvspan(combo.start[i],combo.end[i],facecolor='lightgray',alpha=.5,label='Recession')
    
    #plotting legend and making sure to not include duplicates of 'Recession' label
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    
    #When saving the file to pdf assume that the 'username_q2' folder already exists
    plt.savefig(outputfolder+'mart2336_q2.pdf')


#Q3
# Starting point
def q3():
    # oecd countries and their joining year
    oecd={'country':['Australia','Austria', 'Belgium', 'Canada','Chile','Colombia','Czech Republic','Denmark', 'Estonia',\
                     'Finland', 'France', 'Germany', 'Greece', 'Hungary','Iceland','Ireland','Israel','Italy', 'Japan', \
                     'Korea','Latvia', 'Lithuania','Luxembourg', 'Mexico', 'Netherlands','New Zealand','Norway','Poland',\
                     'Portugal','Slovak Republic','Slovenia','Spain','Sweden','Switzerland','Turkey', 'United Kingdom','United States'],\
          'join_year':[1971,1961, 1961, 1961,2010,2020,1995,1961, 2010, 1969, 1961, 1961, 1961, 1996,\
            1961,1961,2010,1962, 1964,1996,2016, 2018,1961, 1994, 1961,1973,1961,1996,1961,\
            2000,2010,1961,1961,1961,1961,1961,1961]}
    
    #loading gdppercapita data and setting country as index
    gdp_per = pd.read_csv(datafolder+'gdppercapita.csv')
    gdp_per.set_index('country', inplace=True)
    
    #creating list of countries from oecd dictionary
    oecd_countries = list(oecd['country'])
    
    #dropping rows from gdp_per dataframe pertaining to countries that are not oecd countries
    for country in gdp_per.index:
        if country in oecd_countries:
            pass
        else:
            gdp_per = gdp_per.drop(country, axis=0)
    
    #creating list of ordered unique entry years from oecd dictionary
    unique_yrs = sorted(set(list(oecd['join_year'])))
    unique_yrs = list(map(str, unique_yrs))
    
    aggregate_gdp = []
    #creating list containing aggregate gdp per capita average for each unique entry year
    for year in unique_yrs:
        avg = round(gdp_per[str(year)].mean(axis=0),2)
        aggregate_gdp.append(avg)
    
    #plotting entry year vs aggregate gdp per capita
    plt.plot(figsize=(10,5))
    plt.plot(unique_yrs, aggregate_gdp, linewidth=1, marker='o')
    plt.xlabel('Year', fontsize='x-large')
    plt.ylabel('Aggregate GDP Per Capita', fontsize='x-large')
    plt.xticks(unique_yrs)
    plt.xticks(rotation=90)
    
    #When saving the file to pdf assume that the 'username_q3' folder already exists
    plt.savefig(outputfolder+'mart2336_q3.pdf')
